This document parses and transforms CSV High Frontier 4 patent cards into JSON.

# Preamble

## CSV import

In [1]:
import csv
import glob
from os.path import splitext, basename
import unittest

def read_cards(deckname, fields):
    with open(f'decks/{deckname}.csv', 'r') as fh:
        reader = csv.DictReader(fh, fieldnames=fields)
        next(reader) # Drop row 1, which contains multi-collumn cells describing groups of fields (e.g. Thruster and Support Requirements)
        next(reader) # Drop row 2, which contains field names
        deck = list(reader)
        
    for i, card in enumerate(deck):
        split_keys(card)
               
    return deck    

def side_from_index(i):
    if i % 2 == 0:
        return 'Front'
    return 'Reverse'

def split_keys(card):
    namespaced_keys = [k for k in card if '.' in k]
    for key in namespaced_keys:
        k0, k1 = key.split('.')
        if not card.get(k0):
            card[k0] = {}
        card[k0][k1] = card.pop(key)

def chunklist(l, n):
    return zip(*[iter(l)] * 2)
               
class TestUtils(unittest.TestCase):
    def test_split(self):
        card = {
            'mass': 0,
            'radHard': 4,
            'thrustTriangle.thrust': 3,
            'thrustTriangle.fuelConsumption': '2',
            'supportRequirements.⟛ Generator': '1'
        }
        
        expected_result = {
            'mass': 0,
            'radHard': 4,
            'thrustTriangle': {
                'thrust': 3,
                'fuelConsumption': '2',
            },
            'supportRequirements': {'⟛ Generator': '1'},
        }
        
        split_keys(card)
        self.assertEqual(card, expected_result)
        
unittest.main(argv=['', 'TestUtils'], verbosity=2, exit=False)

test_split (__main__.TestUtils) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


## Debug tools

In [2]:
import random

def draw(deck):
    return random.choice(deck)

## Common schema

In [3]:
from jsonschema import validate, ValidationError, RefResolutionError
import json
from os import path, makedirs

import subprocess

SCHEMA_LOCATION = 'http://localhost:8800/'

def save_schema(schema):
    try:
        validate({}, schema)
    except (ValidationError, RefResolutionError) as e:
        pass
    filepath = path.join('schema/', schema['$id'].removeprefix(SCHEMA_LOCATION))
    makedirs(path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w') as f:
        json.dump(schema, f, ensure_ascii=False, indent=2, sort_keys=True)
    
    return schema

thrust_triangle_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'thrust-triangle.schema.json'),
        'title': 'Thrust Triangle',
        'description': 'A thrust triangle on a card.',
        'type': 'object',
        'properties': {
            'thrust': {
                'description': 'The base thrust of the thrust triangle',
                'type': 'integer',
            },
            'fuelConsumption': {
                'description': 'A string representation of the base fuel consumption as an integer or fraction.',
                'type': 'string',
            },
            'fuelType': {
                'description': 'The type of fuel the thrust triangle consumes.',
                'enum': ['water', 'dirt', 'isotope'],
            },
            'afterburn': {
                'description': 'The number of afterburns the thruster can perform.',
                'type': 'integer',
            },
            'push': {
                'description': 'Whether the thrust triangle is pushable.',
                'type': 'boolean',
            },
            'solar': {
                'description': 'Whether the thrust triangle is solar.',
                'type': 'boolean',
            }
        },
        'required': ['thrust', 'fuelConsumption', 'fuelType'],
        'additionalProperties': False,
    }
)

isru_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'isru.schema.json'),
        'title': 'ISRU',
        'description': "The card's ISRU rating and platform icons.",
        'type': 'object',
        'properties': {
            'rating': {
                'description': 'The ISRU rating.',
                'type': 'integer',
            },
            'platforms': {
                'description': 'An array of platforms icons on the card.',
                'type': 'array',
                'items': {
                    'enum': ['raygun', 'buggy', 'missile'],                
                },
            },
        },
        'required': ['rating', 'platforms'],
        'additionalProperties': False,
    }
)

generator_support_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'supports/generator.schema.json'),
        'title': 'Generator Support',
        'properties': {
            'type': { 'const': 'generator' },
            'subtypes': {
                'type': 'array',
                'items': { 'enum': ['e', '⟛'] }
            },
        },
        'required': ['type', 'subtypes'],
        'additionalProperties': False,
    }
)

reactor_support_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'supports/reactor.schema.json'),
        'title': 'Reactor Support',
        'properties': {
            'type': { 'const': 'reactor' },
            'subtypes': {
                'type': 'array',
                'items': { 'enum': ['X', '∿', '💣'] }
            },
        },
        'required': ['type', 'subtypes'],
        'additionalProperties': False,
    }
)

cooling_support_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'supports/cooling.schema.json'),
        'title': 'Cooling Support Requirement',
        'properties': {
            'type': { 'const': 'cooling' },
            'therms': { 'type': 'integer' },
        },
        'required': ['type', 'therms'],
        'additionalProperties': False,
    }
)

support_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'supports/any.schema.json'),
        'title': 'Support',
        'oneOf': [
            { '$ref': generator_support_schema['$id'] },
            { '$ref': reactor_support_schema['$id'] },
            { '$ref': cooling_support_schema['$id'] },
        ]
    }
)

def extend_side_schema(side_schema, schema):
    '''
    Makes schema oneOf the options for side_schema
    '''
    side_schema['oneOf'].append({ '$ref': schema['$id'] })
    return side_schema

null_schema = lambda name: {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    '$id': path.join(SCHEMA_LOCATION, f'{name}.schema.json'),
    'type': 'null',
}

thruster_schema = save_schema(null_schema('thruster'))
robonaut_schema = save_schema(null_schema('robonaut'))
refinery_schema = save_schema(null_schema('refinery'))
reactor_schema = save_schema(null_schema('reactor'))
radiator_schema = save_schema(null_schema('radiator'))
generator_schema = save_schema(null_schema('generator'))
bernal_schema = save_schema(null_schema('bernal'))
human_colonist_schema = save_schema(null_schema('human-colonist'))
gw_thruster_schema = save_schema(null_schema('gw-thruster'))
freighter_schema = save_schema(null_schema('freighter'))
robot_colonist_schema = save_schema(null_schema('robot-colonist'))
tw_thruster_schema = save_schema(null_schema('tw-thruster'))
freighter_fleet_schema = save_schema(null_schema('freighter-fleet'))
promoted_bernal_schema = save_schema(null_schema('promoted-bernal'))
promoted_human_colonist_schema = save_schema(null_schema('promoted-human-colonist'))
promoted_robot_colonist_schema = save_schema(null_schema('promoted-robot-colonist'))

white_side_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'sides/white.schema.json'),
        'title': 'Card White-Side',
        'type': 'object',
        'anyOf': [
            { "$ref": thruster_schema['$id'] },
            { '$ref': robonaut_schema['$id'] },
            { '$ref': refinery_schema['$id'] },
            { '$ref': reactor_schema['$id'] },
            { '$ref': radiator_schema['$id'] },
            { '$ref': generator_schema['$id'] },
            { '$ref': bernal_schema['$id'] },
            { '$ref': human_colonist_schema['$id'] },
        ],
    }
)

black_side_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'sides/black.schema.json'),
        'title': 'Card Black-Side',
        'type': 'object',
        'anyOf': [
            { "$ref": thruster_schema['$id'] },
            { "$ref": robonaut_schema['$id'] },
            { "$ref": refinery_schema['$id'] },
            { "$ref": reactor_schema['$id'] },
            { "$ref": radiator_schema['$id'] },
            { "$ref": generator_schema['$id'] },
            { "$ref": gw_thruster_schema['$id'] },
            { "$ref": freighter_schema['$id'] },
            { "$ref": robot_colonist_schema['$id'] },
        ],
    }
)

purple_side_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'sides/purple.schema.json'),
        'title': 'Card Purple-Side',
        'type': 'object',
        'anyOf': [
            { "$ref": tw_thruster_schema['$id'] },
            { "$ref": freighter_fleet_schema['$id'] },
            { "$ref": promoted_bernal_schema['$id'] },
            { "$ref": promoted_human_colonist_schema['$id'] },
            { "$ref": promoted_robot_colonist_schema['$id'] },
        ],
    }
)

card_schema = save_schema(
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': path.join(SCHEMA_LOCATION, 'card.schema.json'),
        'title': 'Card',
        'type': 'object',
        'properties': {
            'cardId': { 'type': 'integer' },
            'deck': { 'enum': [ 'Thrusters', 'Robonauts', 'Refineries', 'Reactors', 'Radiators', 'Generators', 'GW/TW Thrusters', 'Freighters', 'Bernals', 'Colonists' ] },
            'front': { 'anyOf': [
                { '$ref': white_side_schema['$id'] },
                { '$ref': black_side_schema['$id'] },
            ] },
            'back': { 'anyOf': [
                { '$ref': black_side_schema['$id'] },
                { '$ref': purple_side_schema['$id'] },
            ] },
            'spectralType': {
                'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Any'],
            },
            'promotionColony': {
                'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Submarine', 'Astrobiology', 'Atmospheric', 'Push'],
            },
        },
        'required': ['cardId', 'deck', 'front', 'back'],
        'additionalProperties': False,
    }
)

from jsonschema import validate, ValidationError
def validate_deck(deck, schema):
    for card in deck:
        name = '%s/%s' % (card['front']['name'], card['back']['name'])
        try:
            print('Validating %s: %s' % (card['deck'], name))
            validate(card, schema)
        except (AttributeError, ValidationError) as e:
            print('Error validating %s: %s' % (card['deck'], name))
            print(json.dumps(card, ensure_ascii=False, indent=4))
            raise e
    print('ok')

## Common transformations

In [4]:
import unittest

def reactor_support(*subtypes):
    if all(map(lambda s: s in ['∿', '💣', 'X'], subtypes)):
        return {
            'type': 'reactor',
            'subtypes': list(subtypes), 
        }

def generator_support(*subtypes):
    if all(map(lambda s: s in ['e', '⟛'], subtypes)):
        return {
            'type': 'generator',
            'subtypes': list(subtypes), 
        }

def cooling_support(therms):
    return {
        'type': 'cooling',
        'therms': therms,
    }
    
def get_support_requirements(card):
    supports = []
    
    # Special case for Colliding FRC 3He-D Fusion TeraWatt Thruster
    if card.get('supportRequirements', {}).get('anyReactor', False):
        any_reactor = reactor_support('∿', '💣', 'X')
        supports += [any_reactor, any_reactor]      

    reactor_subtypes = []    
    # Other than the Colliding FRC 3He-D Fusion TeraWatt Thruster, cards only need 1 reactor symbol. If we have multiple symbols, that means that we need any one of them, not several.
    for t in ['∿', '💣', 'X']:
        if card.get('supportRequirements', {}).get(t, False):
            reactor_subtypes.append(t)

    if len(reactor_subtypes) != 0:
        supports.append(reactor_support(*reactor_subtypes))

    # Cards only ever need exactly one generator type
    if card.get('supportRequirements', {}).get('e', False):
        supports.append(generator_support('e'))
    elif card.get('supportRequirements', {}).get('⟛', False):
        supports.append(generator_support('⟛'))

    therms = card.get('supportRequirements', {}).get('cooling', 0)
    if therms != 0:
        supports.append(cooling_support(therms))
    
    return supports

def get_supports(card):
    supports = []
    
    reactor_subtypes = []    
    for t in ['∿', '💣', 'X']:
        if card.get('supports', {}).get(t, False):
            reactor_subtypes.append(t)
    if len(reactor_subtypes) != 0:
        supports.append(reactor_support(*reactor_subtypes))

    generator_subtypes = []
    for t in ['e', '⟛']:
        if card.get('supports', {}).get(t, False):
            generator_subtypes.append(t)
    if len(generator_subtypes) != 0:
        supports.append(generator_support(*generator_subtypes))
        
    therms = card.get('therms', 0)
    if therms != 0:
        supports.append(cooling_support(therms))
    
    return supports
    
class SupportRequirementTests(unittest.TestCase):
    
    def test_no_supports(self):
        card = {}
        self.assertEqual(get_support_requirements(card), [])
        
    def test_single_generator(self):
        card = {'supportRequirements': {'⟛': True}}
        self.assertEqual(get_support_requirements(card), [{'type': 'generator', 'subtypes': ['⟛']}])

    def test_single_reactor(self):
        card = {'supportRequirements': {'💣': True}}
        self.assertEqual(get_support_requirements(card), [{'type': 'reactor', 'subtypes': ['💣']}])
        
    def test_multiple_reactors(self):
        card = {'supportRequirements': {'💣': True, '∿': True, 'X': True}}
        self.assertEqual(get_support_requirements(card), [{
            'type': 'reactor',
            'subtypes': ['∿', '💣', 'X']
        }])

    def test_reactor_and_generator(self):
        card = {'supportRequirements': {'💣': True, 'X': False, 'e': True}}
        self.assertEqual(get_support_requirements(card), [{'type': 'reactor', 'subtypes': ['💣']}, {'type': 'generator', 'subtypes': ['e']}])
        
    def test_therms(self):
        card = {'supportRequirements': {'cooling': 2}}
        self.assertEqual(get_support_requirements(card), [{'type': 'cooling', 'therms': 2}])
    
    def test_colliding_frc_special_case(self):
        '''
        The Colliding FRC 3He-D Fusion GW Thruster has the 'Any' reactor subtype in the spreadsheet. It actually requires TWO of any reactor. We handle this as a special case which requires two Reactor type Support Requirements.
        '''
        card = {'supportRequirements': {'anyReactor': True}}
        any_reactor = {'type': 'reactor', 'subtypes': ['∿', '💣', 'X']}
        self.assertEqual(get_support_requirements(card), [any_reactor, any_reactor])
        
class SupportTests(unittest.TestCase):
    
    def test_no_supports(self):
        card = {}
        self.assertEqual(get_supports(card), [])
        
    def test_simple_reactor(self):
        reactor = {
            'supports': {
                'X': True,
                '💣': False,
                '∿': False,
            }
        }
        self.assertEqual(get_supports(reactor), [reactor_support('X')])
    
    def test_simple_generator(self):
        generator = {
            'supports': {
                'e': True,
                '⟛': False,
            }
        }
        self.assertEqual(get_supports(generator), [generator_support('e')])
        
    def test_radiator(self):
        radiator = {
            'therms': 2
        }
        self.assertEqual(get_supports(radiator), [cooling_support(2)])
        
    def test_multi_reactor(self):
        reactor = {
            'supports': {
                'X': True,
                '💣': True,
                '∿': False,
            }
        }
        self.assertEqual(get_supports(reactor), [reactor_support('💣', 'X')])

    def test_multi_generator(self):
        generator = {
            'supports': {
                'e': True,
                '⟛': True,
            }
        }
        self.assertEqual(get_supports(generator), [generator_support('e', '⟛')])
    
unittest.main(argv=['', 'SupportRequirementTests'], verbosity=2, exit=False)
unittest.main(argv=['', 'SupportTests'], verbosity=2, exit=False)

test_colliding_frc_special_case (__main__.SupportRequirementTests)
The Colliding FRC 3He-D Fusion GW Thruster has the 'Any' reactor subtype in the spreadsheet. It actually requires TWO of any reactor. We handle this as a special case which requires two Reactor type Support Requirements. ... ok
test_multiple_reactors (__main__.SupportRequirementTests) ... ok
test_no_supports (__main__.SupportRequirementTests) ... ok
test_reactor_and_generator (__main__.SupportRequirementTests) ... ok
test_single_generator (__main__.SupportRequirementTests) ... ok
test_single_reactor (__main__.SupportRequirementTests) ... ok
test_therms (__main__.SupportRequirementTests) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.006s

OK
test_multi_generator (__main__.SupportTests) ... ok
test_multi_reactor (__main__.SupportTests) ... ok
test_no_supports (__main__.SupportTests) ... ok
test_radiator (__main__.SupportTests) ... ok
test_simple_generator (__main__.Support

In [5]:
import unittest

def get_isru(robonaut):
    return {
        'rating': robonaut['ISRU']['ISRU'],
        'platforms': [p.lower() for p in ['Raygun', 'Buggy', 'Missile'] if robonaut.get('ISRU').get(p)],
    }

class ISRUTests(unittest.TestCase):
    
    def test_raygun(self):
        card = {'ISRU': {'ISRU': 2, 'Raygun': True, 'Buggy': False, 'Missile': False}}
        self.assertEqual(get_isru(card), {'rating': 2, 'platforms': ['raygun']})
    
    def test_buggy(self):
        card = {'ISRU': {'ISRU': 0, 'Raygun': False, 'Buggy': True, 'Missile': False}}
        self.assertEqual(get_isru(card), {'rating': 0, 'platforms': ['buggy']})
        
    def test_missile(self):
        card = {'ISRU': {'ISRU': 1, 'Raygun': False, 'Buggy': False, 'Missile': True}}
        self.assertEqual(get_isru(card), {'rating': 1, 'platforms': ['missile']})

    def test_multiple(self):
        card = {'ISRU': {'ISRU': 1, 'Raygun': True, 'Buggy': False, 'Missile': True}}
        self.assertEqual(get_isru(card), {'rating': 1, 'platforms': ['raygun', 'missile']})
        
unittest.main(argv=['', 'ISRUTests'], verbosity=2, exit=False)

test_buggy (__main__.ISRUTests) ... ok
test_missile (__main__.ISRUTests) ... ok
test_multiple (__main__.ISRUTests) ... ok
test_raygun (__main__.ISRUTests) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK


In [6]:
import json

def str_to_int(s):
    if s == '':
        return 0
    else:
        return int(s)

def str_to_bool(s):
    if s == '0' or s == '':
        return False
    else:
        return True

def intify(card, fields):
    for k in fields:
        try:
            if '.' in k:
                k0, k1 = k.split('.')
                card[k0][k1] = str_to_int(card[k0][k1])
            else:
                card[k] = str_to_int(card[k])
        except TypeError as e:
            if '.' in k:
                print(f'Could not convert {k0}: {k1} with value {card[k0][k1]} to int')
            else:
                print(f'Could not convert {k} with value {card[k]} to int')
            raise e
            
def boolify(card, fields):
    for k in fields:
        try:
            if '.' in k:
                k0, k1 = k.split('.')
                card[k0][k1] = str_to_bool(card[k0][k1])
            else:
                card[k] = str_to_bool(card[k])
        except TypeError as e:
            if '.' in k:
                print(f'Could not convert {k0}: {k1} with value {card[k0][k1]} to int')
            else:
                print(f'Could not convert {k} with value {card[k]} to int')
            raise e
            
def lowercase(card, fields):
    for k in fields:
        if '.' in k:
            k0, k1 = k.split('.')
            card[k0][k1] = card[k0][k1].lower()
        else:
            card[k] = card[k].lower()
            
def deckname_to_type(deckname):
    return {
        'Bernals': 'bernal',
        'Colonists': 'colonist',
        'Freighters': 'freighter',
        'Generators': 'generator',
        'GW Thrusters': 'gw/tw thrusters',
        'Radiators': 'radiator',
        'Reactors': 'reactor',
        'Refineries': 'refinery',
        'Robonauts': 'robonaut',
        'Thrusters': 'thruster',
    }[deckname]

def normalize_deckname(deckname):
    return {
        'Bernals': 'Bernals',
        'Colonists': 'Colonists',
        'Freighters': 'Freighters',
        'Generators': 'Generators',
        'GW Thrusters': 'GW/TW Thrusters',
        'Radiators': 'Radiators',
        'Reactors': 'Reactors',
        'Refineries': 'Refineries',
        'Robonauts': 'Robonauts',
        'Thrusters': 'Thrusters',
    }[deckname]

def has_empty_thrust_triangle(card):
    thrust_triangle = card.get('thrustTriangle', None)
    if thrust_triangle is not None:
        if thrust_triangle.get('thrust', '') == '':
            return True
    return False

def has_empty_movement_modifiers(card):
    modifiers = card.get('movementModifiers', None)
    if modifiers is not None:
        if modifiers.get('thrust', '') == '':
            return True
    return False

def build_deck(deckname, fields, int_fields, bool_fields, uppercased_fields):
    deck = []
    for front, back in chunklist(read_cards(deckname, fields), 2):
        try:
            spectral_type = front.get('spectralType', None)
            if spectral_type == 'n/a':
                spectral_type = None
            
            promotion_colony = front.get('promotionColony', None)
            front.pop('future', None)

            for side in [front, back]:
                side.pop('spectralType', None)
                side.pop('promotionColony', None)
                
                # Only colonists are allowed types
                if side.get('type', None) not in ['Engineer', 'Prospector', 'Industrialist', 'Miner']:
                    side.pop('type', None)
                
                omit_fields = []
                if has_empty_thrust_triangle(side):
                    side.pop('thrustTriangle')
                    omit_fields += filter(lambda x: x.startswith('thrustTriangle.'), fields)

                if has_empty_thrust_triangle(side):
                    side.pop('movementModifiers')
                    omit_fields += filter(lambda x: x.startswith('movementModifiers.'), fields)

                intify(side, [f for f in int_fields if f not in omit_fields])
                boolify(side, [f for f in bool_fields if f not in omit_fields])
                lowercase(side, [f for f in uppercased_fields if f not in omit_fields])

                support_requirements = get_support_requirements(side)
                if len(support_requirements) > 0:
                    side['supportRequirements'] = support_requirements
                else:
                    side.pop('supportRequirements', None)

                supports = get_supports(side)
                if len(supports) > 0:
                    side['supports'] = supports
                else:
                    side.pop('supports', None)

                isru = side.get('ISRU', None)
                if isru:
                    side['ISRU'] = get_isru(side)
                    
                if side.get('ability', None) == '':
                    side.pop('ability')

            card = {
                'cardId': 0, # placeholder
                'deck': normalize_deckname(deckname),
                'front': front,
                'back': back,
            }
                            
            if spectral_type:
                card['spectralType'] = spectral_type
            if promotion_colony:
                card['promotionColony'] = promotion_colony
            
            deck.append(card)
            
        except Exception as e:
            print(json.dumps(front, ensure_ascii=False, indent=2))
            print(json.dumps(back, ensure_ascii=False, indent=2))
            raise e
    return deck

# Thrusters

## Import

In [7]:
'''
Thrusters
'''

fields = [
    'name',
    'spectralType',
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.fuelConsumption',
    'thrustTriangle.fuelType',
    'bonusPivots',
    'thrustTriangle.afterburn',
    'thrustTriangle.push',
    'thrustTriangle.solar',
    'supportRequirements.⟛',
    'supportRequirements.e',
    'supportRequirements.X',
    'supportRequirements.∿',
    'supportRequirements.💣',
    'supportRequirements.cooling',
    'ability',
]

int_fields = [
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.afterburn',
    'supportRequirements.cooling',
    'bonusPivots',
]

bool_fields = [
    'thrustTriangle.push',
    'thrustTriangle.solar',
    'supportRequirements.⟛',
    'supportRequirements.e',
    'supportRequirements.X',
    'supportRequirements.∿',
    'supportRequirements.💣',
]

uppercased_fields = [
    'thrustTriangle.fuelType',
]

thrusters = build_deck('Thrusters', fields, int_fields, bool_fields, uppercased_fields)

## Validation

In [8]:
import json
from os import path

thruster_schema = save_schema(thruster_schema | {
    'title': 'Thruster',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
        'supportRequirements': {
            'type': 'array',
            'items': { '$ref': support_schema['$id'] }
        },
        'bonusPivots' : { 'type': 'integer' },
        'ability': { 'type': 'string' },
    },
    'required': ['name', 'mass', 'radHard', 'thrustTriangle'],
    'additionalProperties': False,
})

validate_deck(thrusters, card_schema)

Validating Thrusters: Ablative Plate/Ablative Nozzle
Validating Thrusters: De Laval Nozzle/Magnetic Nozzle
Validating Thrusters: Dumbo/Timberwind
Validating Thrusters: Hall Effect/Ion Drive
Validating Thrusters: Mass Driver/MPD T-wave
Validating Thrusters: Metastable Helium/n-6Li Microfission
Validating Thrusters: Monoatomic Plug Nozzle/Vortex Confined Nozzle
Validating Thrusters: Photon Heliogyro/Electric Sail
Validating Thrusters: Photon Kite Sail/Mag Sail
Validating Thrusters: Ponderomotive VASIMR/Pulsed Plasmoid
Validating Thrusters: Pulsed Inductive/Dual-Stage 4-Grid
Validating Thrusters: Re Solar Moth/Colliding Beam H-B Fusion
ok


# Robonauts

In [9]:
'''
Robonauts
'''

fields = [
    'name',
    'spectralType',
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.fuelConsumption',
    'thrustTriangle.fuelType',
    'thrustTriangle.afterburn',
    'thrustTriangle.push',
    'thrustTriangle.solar',
    'ISRU.ISRU',
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'supportRequirements.⟛',
    'supportRequirements.e',
    'supportRequirements.X',
    'supportRequirements.∿',
    'supportRequirements.💣',
    'supportRequirements.cooling',
    'ability',
]

int_fields = [
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.afterburn',
    'supportRequirements.cooling',
    'ISRU.ISRU',
]

bool_fields = [
    'thrustTriangle.push',
    'thrustTriangle.solar',
    'supportRequirements.⟛',
    'supportRequirements.e',
    'supportRequirements.X',
    'supportRequirements.∿',
    'supportRequirements.💣',
    'ISRU.Missile',
    'ISRU.Buggy',
    'ISRU.Raygun',
]

uppercased_fields = [
    'thrustTriangle.fuelType',
]

robonauts = build_deck('Robonauts', fields, int_fields, bool_fields, uppercased_fields)

## Validation

In [10]:
from jsonschema import validate, ValidationError
from os import path
import json

robonaut_schema = save_schema(robonaut_schema | {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    '$id': path.join(SCHEMA_LOCATION, 'robonaut.schema.json'),
    'title': 'Robonaut',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
        'supportRequirements': {
            'type': 'array',
            'items': { '$ref': support_schema['$id'] }
        },
        'ISRU' : { '$ref': isru_schema['$id'] },
        'ability': { 'type': 'string' },
    },
    'required': ['name', 'mass', 'radHard', 'ISRU'],
    'additionalProperties': False,
})

validate_deck(robonauts, card_schema)

Validating Robonauts: Blackbody-Pumped Laser/Fissile Aerosol Laser
Validating Robonauts: Cat Fusion Z-pinch Torch/H-B Cat Inertial
Validating Robonauts: Flywheel Tractor/Electrophoretic Sandworm
Validating Robonauts: Free Electron Laser/Wakefield e-Beam
Validating Robonauts: Kuck Mosquito/Ablative Laser
Validating Robonauts: MET Steamer/Nanobot
Validating Robonauts: Neutral Beam/D-D Fusion Inertial
Validating Robonauts: Nuclear Drill/Helical Railgun
Validating Robonauts: Phase-Locked Diode Laser/Lorentz-Propelled Microprobe
Validating Robonauts: Rock Splitter/MagBeam
Validating Robonauts: Solar-Pumped MHD Exciplex Laser/Quantum Cascade Laser
Validating Robonauts: Tungsten Resistojet/MITEE Arcjet
ok


# Refineries

In [11]:
'''
Refineries
'''

fields = [
    'name',
    'spectralType',
    'mass',
    'radHard',
    'airEater',
    'supportRequirements.e',
    'supportRequirements.X',
    'supportRequirements.∿',
    'supportRequirements.💣',
    'ability',
]

int_fields = [
    'mass',
    'radHard',
]

bool_fields = [
    'airEater',
    'supportRequirements.e',
    'supportRequirements.X',
    'supportRequirements.∿',
    'supportRequirements.💣',
]

uppercased_fields = []

refineries = build_deck('Refineries', fields, int_fields, bool_fields, uppercased_fields)

## Validation

In [12]:
from jsonschema import validate, ValidationError
import json

refinery_schema = save_schema(refinery_schema | {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    '$id': path.join(SCHEMA_LOCATION, 'refinery.schema.json'),
    'title': 'Refinery',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'supportRequirements': {
            'type': 'array',
            'items': { '$ref': support_schema['$id'] },
        },
        'airEater' : { 'type': 'boolean' },
        'ability': { 'type': 'string' },
    },
    'required': ['name', 'mass', 'radHard'],
    'additionalProperties': False,
})

validate_deck(refineries, card_schema)

Validating Refineries: Atomic Layer Deposition/Ilmenite Semiconductor Film
Validating Refineries: Basalt Fiber Spinning/Von Neumann Santa Claus Machine
Validating Refineries: CVD Molding/Carbonyl Volatilization
Validating Refineries: Carbo-Chlorination/Solar Carbotherm
Validating Refineries: Electroforming/Impact Mold Sinter
Validating Refineries: Fluidized Bed/Atmospheric Scoop
Validating Refineries: Foamglass Sintering/Laser-Heated Pedestal Growth
Validating Refineries: Froth Flotation/Femtochemistry
Validating Refineries: ISRU Sabatier/Biophytolytic Algal Farm
Validating Refineries: In-Situ Leaching/Termite Nest
Validating Refineries: Magma Electrolysis/Ionosphere Lasing
Validating Refineries: Supercritical Drying/Solid Flame
ok


# Reactors

In [13]:
'''
Reactors
'''

fields = [
    'name',
    'spectralType',
    'mass',
    'radHard',
    'supports.X',
    'supports.∿',
    'supports.💣',
    'movementModifiers.thrust',
    'movementModifiers.fuelConsumption',
    'airEater',
    'supportRequirements.⟛',
    'supportRequirements.X',
    'supportRequirements.cooling',
    'ability',
]

int_fields = [
    'mass',
    'radHard',
    'movementModifiers.thrust',
    'supportRequirements.cooling',
]

bool_fields = [
    'supports.X',
    'supports.∿',
    'supports.💣',
    'airEater',
    'supportRequirements.⟛',
    'supportRequirements.X',
]

uppercased_fields = []

reactors = build_deck('Reactors', fields, int_fields, bool_fields, uppercased_fields)

## Validation

In [14]:
from jsonschema import validate, ValidationError
import json
from os import path

movement_modifier_schema = save_schema({
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    '$id': path.join(SCHEMA_LOCATION, 'movement-modifiers.schema.json'),
    'title': 'Movement Modifiers',
    'description': 'A movement modifier triangle on a card.',
    'type': 'object',
    'properties': {
        'thrust': {
            'description': 'The thrust modifier',
            'type': 'integer',
        },
        'fuelConsumption': {
            'description': 'A string representation of the fuel consumption modifier as an integer or fraction.',
            'type': 'string',
        },
        'solar': {
            'description': 'Whether the modifier applies solar.',
            'type': 'boolean',
        },
    },
    'required': ['thrust', 'fuelConsumption'],
    'additionalProperties': False,
})

reactor_schema = save_schema(reactor_schema | {
    'title': 'Reactor',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'supports': {
            'type': 'array',
            'items': { '$ref': reactor_support_schema['$id'] },
        },
        'supportRequirements': {
            'type': 'array',
            'items': { '$ref': support_schema['$id'] },
        },
        'movementModifiers': { '$ref': movement_modifier_schema['$id'] },
        'airEater': { 'type': 'boolean' },
        'ability': { 'type': 'string' },
    },
    'required': ['name', 'mass', 'radHard', 'supports'],
    'additionalProperties': False,
})

validate_deck(reactors, card_schema)

Validating Reactors: Cermet NERVA Fission/Pulsed NTR Fission
Validating Reactors: D-D Fusion Magneto-Inertial/H-B Fusion Reciprocating Plasmoid
Validating Reactors: D-T Fusion Tokamak/Antimatter GDM
Validating Reactors: D-T Gun Fusion/Macron Blowpipe Fusion
Validating Reactors: Lyman Alpha Trap/Free Radical Hydrogen Trap
Validating Reactors: Metallic Hydrogen/Fission-Augmented D-T Inertial Fusion
Validating Reactors: Mini-Mag RF Paul Trap/Ultracold Neutrons
Validating Reactors: Pebble Bed Fission/VCR Light Bulb Fission
Validating Reactors: Penning Trap/3He-D Fusion Mirror Cell
Validating Reactors: Project Orion/Project Valkyrie
Validating Reactors: Rubbia Thin Film Fission Hohlraum/Positronium Bottle
Validating Reactors: Supercritical Water Fission/H-6Li Fusor
ok


# Radiators

In [15]:
'''
Radiators
'''

fields = [
    'name',
    'spectralType',
    'lightSide.mass',
    'lightSide.radHard',
    'lightSide.therms',
    'heavySide.mass',
    'heavySide.radHard',
    'heavySide.therms',
    'supportRequirements.e',
    'ability',
]

int_fields = [
    'lightSide.mass',
    'lightSide.radHard',
    'lightSide.therms',
    'heavySide.mass',
    'heavySide.radHard',
    'heavySide.therms',
]

bool_fields = [
    'supportRequirements.e',
]

uppercased_fields = []

radiators = build_deck('Radiators', fields, int_fields, bool_fields, uppercased_fields)

for radiator in radiators:
    for side in [radiator['front'], radiator['back']]:
        support_requirements = side.pop('supportRequirements', None)
        # The spreadsheet has a single support requirements field but technically the light and heavy side requirements are distinct
        # This only affects Magnetocaloric Refrigerator, which has the same support requirement on each side, so we just duplicate it
        if support_requirements:
            side['lightSide']['supportRequirements'] = support_requirements
            side['heavySide']['supportRequirements'] = support_requirements
        
        ability = side.pop('ability', '')
        if ability != '':
            # Li Heatsink Fountain/Thermochemical Heatsink Fountain
            if ability.startswith('[Heavy] '):
                side['heavySide']['ability'] = ability.removeprefix('[Heavy] ')
            # Ignoring the non-existent case of light-side only abilities
            else:
                side['lightSide']['ability'] = ability
                side['heavySide']['ability'] = ability

## Validation

In [16]:
from jsonschema import validate, ValidationError
import json

radiator_light_side = save_schema({
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    '$id': path.join(SCHEMA_LOCATION, 'radiator-sides/light.schema.json'),
    'title': 'Radiator Light Side',
    'type': 'object',
    'properties': {
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'therms': { 'type': 'integer' },
        'supportRequirements': {
            'type': 'array',
            'items': { '$ref': support_schema['$id'] }
        },
        'ability': {'type': 'string'},
    },
    'required': ['mass', 'radHard', 'therms'],
    'additionalProperties': False
})

radiator_heavy_side = save_schema({
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    '$id': path.join(SCHEMA_LOCATION, 'radiator-sides/heavy.schema.json'),
    'title': 'Radiator Heavy Side',
    'type': 'object',
    'properties': {
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'therms': { 'type': 'integer' },
        'supportRequirements': {
            'type': 'array',
            'items': { '$ref': support_schema['$id'] }
        },
        'ability': {'type': 'string'},
    },
    'required': ['mass', 'radHard', 'therms'],
    'additionalProperties': False
})

radiator_schema = save_schema(radiator_schema | {
    'title': 'Radiator',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'lightSide': { '$ref': radiator_light_side['$id'] },
        'heavySide': { '$ref': radiator_heavy_side['$id'] },
    },
    'required': ['name', 'lightSide', 'heavySide'],
    'additionalProperties': False,
})

validate_deck(radiators, card_schema)

Validating Radiators: Bubble Membrane/Electrostatic Membrane
Validating Radiators: Dielectric X-Ray Window/Graphene Crystal X-Ray Mirror
Validating Radiators: ETHER Charged Dust/Curie Point
Validating Radiators: Li Heatsink Fountain/Thermochemical Heatsink Fountain
Validating Radiators: Magnetocaloric Refrigerator/Nuclear Fuel Spin Polarizer
Validating Radiators: Microtube Array/Marangoni Flow
Validating Radiators: Mo / Li Heat Pipe/Tin Droplet
Validating Radiators: Qu Tube/ANDR / In Dream Pipe
Validating Radiators: SS / NaK Pumped Loop/Hula-Hoop
Validating Radiators: Salt-Cooled Reflux Tube/Buckytube Filament
Validating Radiators: Steel / Pb-Bi Pumped Loop/Pulsating Heat Pipe
Validating Radiators: Ti / K Heat Pipe/Flux-Pinned Superthermal
ok


# Generators

In [17]:
'''
Generators
'''

fields = [
    'name',
    'spectralType',
    'mass',
    'radHard',
    'supports.⟛',
    'supports.e',
    'movementModifiers.thrust',
    'movementModifiers.fuelConsumption',
    'airEater',
    'movementModifiers.solar',
    'supportRequirements.e',
    'supportRequirements.X',
    'supportRequirements.∿',
    'supportRequirements.💣',
    'supportRequirements.cooling',
    'ability',
]

int_fields = [
    'mass',
    'radHard',
    'movementModifiers.thrust',
    'supportRequirements.cooling',
]

bool_fields = [
    'supports.⟛',
    'supports.e',
    'airEater',
    'movementModifiers.solar',
    'supportRequirements.e',
    'supportRequirements.X',
    'supportRequirements.∿',
    'supportRequirements.💣',
]

uppercased_fields = []

generators = build_deck('Generators', fields, int_fields, bool_fields, uppercased_fields)

## Validation

In [18]:
from jsonschema import validate, ValidationError
import json
from os import path

generator_schema = save_schema(generator_schema | {
    'title': 'Generator',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'supports': {
            'type': 'array',
            'items': { '$ref': generator_support_schema['$id'] },
        },
        'supportRequirements': {
            'type': 'array',
            'items': { '$ref': support_schema['$id'] },
        },
        'movementModifiers': { '$ref': movement_modifier_schema['$id'] },
        'airEater': { 'type': 'boolean' },
        'ability': { 'type': 'string' },
    },
    'required': ['name', 'mass', 'radHard', 'supports'],
    'additionalProperties': False,
})

import jsonschema
for card in generators:
    print(jsonschema.validate(card['front'], generator_schema))
    break

validate_deck(generators, card_schema)

None
Validating Generators: AMTEC Thermoelectric/JTEC H2 Thermoelectric
Validating Generators: Brayton Turbine/O'Meara LSP Paralens
Validating Generators: Cascade Photovoltaic/Buckyball C60 Photovoltaic
Validating Generators: Cascade Thermoacoustic/Dusty Plasma MHD
Validating Generators: Catalyzed Fission Scintillator/Diamonoid Electro-Dynamic Tether
Validating Generators: Ericsson Engine/Nanocomposite Thermoelectric
Validating Generators: Flywheel Compulsator/Superconducting Adductor
Validating Generators: H2-O2 Fuel Cell/Microbial Fuel Cell
Validating Generators: In-Core Thermionic/Z-Pinch Microfission
Validating Generators: Magnetoshell Plasma Parachute/Granular Rainbow Corral
Validating Generators: Marx Capacitor Bank/Casimir Battery
Validating Generators: Optoelectric Nuclear Battery/Nuclear-Pumped Excimer Flashlamp
Validating Generators: Photon Tether Rectenna/Palmer LSP Aerosol Lens
Validating Generators: Radioisotope Stirling/Triggered Decay Nuclear Battery
Validating Generator

# GW Thrusters

In [19]:
'''
GW Thrusters
'''

fields = [
    'name',
    'type',
    'spectralType',
    'promotionColony',
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.fuelConsumption',
    'thrustTriangle.afterburn',
    'supportRequirements.⟛',
    'supportRequirements.e',
    'supportRequirements.X',
    'supportRequirements.any',
    'supportRequirements.cooling',
    'future',
]

int_fields = [
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'supportRequirements.cooling',
    'thrustTriangle.afterburn',
]

bool_fields = [
    'supportRequirements.⟛',
    'supportRequirements.e',
    'supportRequirements.X',
    'supportRequirements.any',
]

uppercased_fields = []

gwthrusters = build_deck('GW Thrusters', fields, int_fields, bool_fields, uppercased_fields)

for patent in gwthrusters:
    for side in [patent['front'], patent['back']]:
        side['thrustTriangle']['fuelType'] = 'isotope'

## Validation

In [20]:
from jsonschema import validate, ValidationError
import json

gw_thruster_schema = save_schema(gw_thruster_schema | {
    'title': 'GW Thruster',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
        'supportRequirements': {
            'type': 'array',
            'items': { '$ref': support_schema['$id'] },
        },
        'ability': { 'type': 'string' },
    },
    'required': ['name', 'mass', 'radHard', 'thrustTriangle'],
    'additionalProperties': False,
})

tw_thruster_schema = save_schema(tw_thruster_schema | {
    'description': "The TW Thruster side of a card from the GW/TW Thruster deck.",
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
        'supportRequirements': {
            'type': 'array',
            'items': { '$ref': support_schema['$id'] },
        },
        'ability': { 'type': 'string' },
        'future': {'type': 'string' },
    },
    'required': ['name', 'mass', 'radHard', 'thrustTriangle', 'future'],
    'additionalProperties': False,
})

validate_deck(gwthrusters, card_schema)

Validating GW/TW Thrusters: Amat-Catalyzed Fission-Fusion/Amat-Initiated H-B Magnetic-Inertial
Validating GW/TW Thrusters: Dense Plasma H-B Focus Fusion/Crossfire H-B Focus Fusion
Validating GW/TW Thrusters: Levitated Dipole 6Li-H Fusion/Dusty Plasma
Validating GW/TW Thrusters: Mini-Mag Orion Z-Pinch Fission/Solem Medusa Tugged Orion
Validating GW/TW Thrusters: Salt-Water Zubrin/Zubrin-GDM
Validating GW/TW Thrusters: Spheromak 3He-D Magnetic Fusion/Colliding FRC 3He-D Fusion
Validating GW/TW Thrusters: VISTA D-T Inertial Fusion/Daedalus 3He-D Inertial Fusion
ok


# Freighters

In [21]:
'''
Freighters
'''

fields = [
    'name',
    'type',
    'spectralType',
    'promotionColony',
    'mass',
    'radHard',
    'loadLimit',
    'factoryLoadingOnly',
    'bonusPivots',
    'supports.⟛',
    'supports.e',
    'supports.X',
    'supports.∿',
    'ability',
    'future',
]

int_fields = [
    'mass',
    'radHard',
    'loadLimit',
    'bonusPivots',
]

bool_fields = [
    'factoryLoadingOnly',
    'supports.⟛',
    'supports.e',
    'supports.X',
    'supports.∿',
]

freighters = build_deck('Freighters', fields, int_fields, bool_fields, [])

## Validation

In [22]:
from jsonschema import validate, ValidationError
import json

freighter_schema = save_schema(freighter_schema | {
    'title': 'Freighter',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'supports': {
            'type': 'array',
            'item': { '$ref': support_schema['$id'] },
        },
        'loadLimit': { 'type': 'integer' },
        'factoryLoadingOnly': { 'type': 'boolean' },
        'bonusPivots': { 'type': 'integer' },
        'ability': { 'type': 'string' },
    },
    'required': ['name', 'mass', 'radHard', 'loadLimit'],
    'additionalProperties': False,
})

freighter_fleet_schema = save_schema(freighter_fleet_schema | {
    'title': 'Freighter Fleet',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'supports': {
            'type': 'array',
            'item': { '$ref': support_schema['$id'] },
        },
        'loadLimit': { 'type': 'integer' },
        'factoryLoadingOnly': { 'type': 'boolean' },
        'bonusPivots': { 'type': 'integer' },
        'ability': { 'type': 'string' },
        'future': { 'type': 'string' },
    },
    'required': ['name', 'mass', 'radHard', 'loadLimit', 'future'],
    'additionalProperties': False,
})

validate_deck(freighters, card_schema)

Validating Freighters: Fission-Heated Steam/Fission GCR
Validating Freighters: Fusion Fragment Sail/Antiproton Sail and Harvester
Validating Freighters: HIIPER Beam Rider/Magnetic Mirror Beam Rider
Validating Freighters: Inflatable Solar-Heated/Archimedes Palmer Lens
Validating Freighters: Poodle Steam/D-Nanotube Dirt Launcher
Validating Freighters: Rotary Dirt Launcher/KESTS Hoop Dirt Launcher
Validating Freighters: Z-Pinch D-T / 6Li Fusion/Z-Pinch 3He-D Target Fusion
ok


# Bernals

In [23]:
'''
Bernals
'''

fields = [
    'name',
    'type',
    'promotionColony',
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.fuelConsumption',
    'thrustTriangle.push',
    'supportRequirements.e',
    'supportRequirements.cooling',
    'ability',
]

int_fields = [
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'supportRequirements.cooling',
]

bool_fields = [
    'thrustTriangle.push',
    'supportRequirements.e',
]

bernals = build_deck('Bernals', fields, int_fields, bool_fields, [])

for card in bernals:
    for side in [card['front'], card['back']]:
        side['thrustTriangle']['fuelType'] = 'dirt'

## Validate

In [24]:
from jsonschema import validate, ValidationError
import json

bernal_schema = save_schema(bernal_schema | {
    'title': 'Bernal',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
        'supportRequirements': {
            'type': 'array',
            'item': { '$ref': support_schema['$id'] },
        },
        'ability': { 'type': 'string' },
    },
    'required': ['name', 'mass', 'radHard', 'thrustTriangle'],
    'additionalProperties': False,
})

validate_deck(bernals, card_schema)

Validating Bernals: GEO Elevator Bernal/Space Elevator Lab
Validating Bernals: L1 Climate Control Bernal/Climate Control Lab
Validating Bernals: L2 Collimator Bernal/Collimator Lab
Validating Bernals: L3 Lofstrom Loop Microgravity/Lofstrom Loop Microgravity Lab
Validating Bernals: L4 Antimatter Factory/Antimatter Lab
Validating Bernals: L4s Pharmaceutics Bernal/Pharmaceutics Lab
Validating Bernals: L5 Solar Cell Factory/Solar Cell Lab
Validating Bernals: L5s Cancer Hospital/Cancer Lab
Validating Bernals: SSO Diplomatic/Diplomatic Lab
Validating Bernals: Tourism Cycler/Tourism Hotel
ok


# Colonists

In [25]:
'''
Colonists
'''

from functools import reduce

fields = [
    'name',
    'type',
    'specialty',
    'spectralType',
    'promotionColony',
    'ideology',
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.fuelConsumption',
    'thrustTriangle.fuelType',
    'thrustTriangle.afterburn',
    'thrustTriangle.push',
    'thrustTriangle.solar',
    'airEater',
    'bonusPivots',
    'ISRU.ISRU',
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'supports.X',
    'supports.∿',
    'supports.💣',
    'ability',
    'future',
]

int_fields = [
    'mass',
    'radHard',
    'thrustTriangle.thrust',
    'thrustTriangle.afterburn',
    'ISRU.ISRU',
    'bonusPivots',
]

bool_fields = [
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'thrustTriangle.push',
    'thrustTriangle.solar',
    'airEater',
    'supports.X',
    'supports.∿',
    'supports.💣',
]

def map_ideology(color):
    return {
        'Yellow': 'Unity',
        'Red': 'Freedom',
        'Purple': 'Authority',
        'Grey': 'Individuality',
        'Green': 'Equality',
        'White': 'Honor',
    }[color]

colonists = build_deck('Colonists', fields, int_fields, bool_fields, ['thrustTriangle.fuelType'])
for c in colonists:
    ideology = c['front']['ideology']
    if ideology == 'n/a':
        c['front'].pop('ideology')
        c['back'].pop('ideology')
    else:
        c['front']['ideology'] = map_ideology(ideology)
        c['back']['ideology'] = map_ideology(ideology)
        
    specialty = c['front']['specialty']
    c['back']['specialty'] = specialty

In [26]:
from jsonschema import validate, ValidationError
import json

human_colonist_schema = save_schema(human_colonist_schema | {
    'title': 'Human Colonist',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'specialty': { 'enum': ['Engineer', 'Miner', 'Prospector', 'Industrialist'] },
        'ideology': { 'enum': ['Honor', 'Freedom', 'Individuality', 'Equality', 'Authority', 'Unity', 'Centrist'] },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
        'airEater': { 'type': 'boolean' },
        'bonusPivots': { 'type': 'integer' },
        'ISRU': { '$ref': isru_schema['$id'] },
        'supports': {
            'type': 'array',
            'item': { '$ref': support_schema['$id'] },
        },
        'ability': { 'type': 'string' },
    },
    'required': ['name', 'specialty', 'ideology', 'mass', 'radHard'],
    'additionalProperties': False,
})

robot_colonist_schema = save_schema(robot_colonist_schema | {
    'title': 'Robot Colonist',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'specialty': { 'enum': ['Engineer', 'Miner', 'Prospector', 'Industrialist'] },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
        'airEater': { 'type': 'boolean' },
        'bonusPivots': { 'type': 'integer' },
        'ISRU': { '$ref': isru_schema['$id'] },
        'supports': {
            'type': 'array',
            'item': { '$ref': support_schema['$id'] },
        },
        'ability': { 'type': 'string' },
    },
    'required': ['name', 'specialty', 'mass', 'radHard'],
    'additionalProperties': False,
})

promoted_human_colonist_schema = save_schema(promoted_human_colonist_schema | {
    'title': 'Promoted Human Colonist',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'specialty': { 'enum': ['Engineer', 'Miner', 'Prospector', 'Industrialist'] },
        'ideology': { 'enum': ['Honor', 'Freedom', 'Individuality', 'Equality', 'Authority', 'Unity', 'Centrist'] },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
        'airEater': { 'type': 'boolean' },
        'bonusPivots': { 'type': 'integer' },
        'ISRU': { '$ref': isru_schema['$id'] },
        'supports': {
            'type': 'array',
            'item': { '$ref': support_schema['$id'] },
        },
        'ability': { 'type': 'string' },
        'future': { 'type': 'string' },
    },
    'required': ['name', 'specialty', 'ideology', 'mass', 'radHard', 'future'],
    'additionalProperties': False,
})

promoted_robot_colonist_schema = save_schema(promoted_robot_colonist_schema | {
    'title': 'Promoted Robot Colonist',
    'type': 'object',
    'properties': {
        'name': { 'type': 'string' },
        'specialty': { 'enum': ['Engineer', 'Miner', 'Prospector', 'Industrialist'] },
        'mass': { 'type': 'integer' },
        'radHard': { 'type': 'integer' },
        'thrustTriangle': { '$ref': thrust_triangle_schema['$id'] },
        'airEater': { 'type': 'boolean' },
        'bonusPivots': { 'type': 'integer' },
        'ISRU': { '$ref': isru_schema['$id'] },
        'supports': {
            'type': 'array',
            'item': { '$ref': support_schema['$id'] },
        },
        'ability': { 'type': 'string' },
        'future': { 'type': 'string' },
    },
    'required': ['name', 'specialty', 'mass', 'radHard', 'future'],
    'additionalProperties': False,
})

validate_deck(colonists, card_schema)

Validating Colonists: Babbage Halbonauts/Utility Fog Halbonaut
Validating Colonists: Biomechs/Group Mind Immortalists
Validating Colonists: Botany Bay Convicts/Soldier Caste
Validating Colonists: Boyle Engineering Collective/Martian Assembly
Validating Colonists: Calypso 2 Seed Sail/Wet-Nano Seed Sail
Validating Colonists: Heavy Water Survivalists/New Attica Secessionists
Validating Colonists: House of Saud/Iceworms
Validating Colonists: Juiced Cosmonauts/Rental Body Guild
Validating Colonists: Lloyd's Salvage Co./Svalbard Caretakers
Validating Colonists: Malcolm/Renaissance Man
Validating Colonists: Microgravity Pantrophists/Blue Goo Sybonts
Validating Colonists: Programmable Matter/Neumann Matter
Validating Colonists: Rock Rats Miners' Union/Alchemist Aviatrices
Validating Colonists: Security System/Frankenstein Navigator
Validating Colonists: Siren Cybernautics Inc./Josephson Implants
Validating Colonists: Smart Pets/Creeper Neogen
Validating Colonists: Transorbital Railworkers/Kalu

# Validation

## Aggregating card lists

In [27]:
from itertools import chain

all_cards = list(chain(thrusters, robonauts, refineries, reactors, radiators, generators, gwthrusters, freighters, bernals, colonists))

# Output

In [29]:
from itertools import chain
import json

card_sets = {
    'thrusters': thrusters,
    'robonauts': robonauts,
    'refineries': refineries,
    'reactors': reactors,
    'radiators': radiators,
    'generators': generators,
    'gw-tw-thrusters': gwthrusters,
    'freighters': freighters,
    'bernals': bernals,
    'colonists': colonists,
}

for name, deck in card_sets.items():
    print(f'Writing {name}.json')
    with open(f'json/{name}.json', 'w') as f:
        json.dump(deck, f, ensure_ascii=False, indent=2)

Writing thrusters.json
Writing robonauts.json
Writing refineries.json
Writing reactors.json
Writing radiators.json
Writing generators.json
Writing gw-tw-thrusters.json
Writing freighters.json
Writing bernals.json
Writing colonists.json
